<a href="https://colab.research.google.com/github/voilszee/kelompok-11-apm/blob/main/single_layer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import PCA as sklearnPCA

# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

In [ ]:
wbcd = pd.read_csv("../input/breast-cancer-wisconsin-data")
wbcd.head()

In [ ]:
wbcd = wbcd.replace('?', 0)

In [ ]:
print("This WBCD dataset is consisted of",wbcd.shape)

In [ ]:
sns.countplot(wbcd['class'],label="Count")

In [ ]:
train,test = train_test_split(wbcd, test_size=0.3, random_state=42)
print("Training Data :",train.shape)
print("Testing Data :",test.shape)

In [ ]:
train_data = train
test_data = test


print("Training Data :",train_data.shape)
print("Testing Data :",test_data.shape)

In [ ]:
train.iloc[:,0:-1].head()

In [ ]:
train_x = train_data.iloc[:,0:-1]
train_x = MinMaxScaler().fit_transform(train_x)
print("Training Data :", train_x.shape)

# Testing Data
test_x = test_data.iloc[:,0:-1]
test_x = MinMaxScaler().fit_transform(test_x)
print("Testing Data :", test_x.shape)


In [ ]:
train_y = train_data.iloc[:,-1:]
train_y[train_y== 2] = 0
train_y[train_y== 4 ] = 1
print("Training Data :", train_y.shape)

# Testing Data
test_y = test_data.iloc[:,-1:]
test_y[test_y== 2] = 0
test_y[test_y==4 ] = 1
print("Testing Data :", test_y.shape)

In [ ]:
X = tf.placeholder(tf.float32, [None,9])
Y = tf.placeholder(tf.float32, [None, 1])

In [ ]:
# weight
W = tf.Variable(tf.random_normal([9,1], seed=0), name='weight')

# bias
b = tf.Variable(tf.random_normal([1], seed=0), name='bias')

In [ ]:
logits = tf.matmul(X,W) + b

In [ ]:
hypothesis = tf.nn.sigmoid(logits)

cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
cost = tf.reduce_mean(cost_i)
# cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

In [ ]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [ ]:
prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: train_x, Y: train_y})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
    test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
    print("Model Prediction =", train_acc)
    print("Test Prediction =", test_acc)